# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [22]:
# import libraries
import pandas as pd 
from sqlalchemy import create_engine
import re
import nltk
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('wordnet')

from nltk.corpus import stopwords
nltk.download('stopwords')

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.multioutput import MultiOutputClassifier

from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/harryashby/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/harryashby/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/harryashby/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [45]:
# load data from database
engine = create_engine('sqlite:///dbtest.db')
df = pd.read_sql('disaster_messages', engine)
df
X = df.message
Y = df.drop(columns = ['message', 'original', 'genre','id'])

### 2. Write a tokenization function to process your text data

In [62]:
stopwords_eng = stopwords.words("english")
lemmatizer = WordNetLemmatizer()


def tokenize(text):
     # Replace all urls with a urlplaceholder string
    url_regex = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # Extract all the urls from the provided text 
    detected_urls = re.findall(url_regex, text)
    # Replace url with a url placeholder string
    for detected_url in detected_urls:
        text = text.replace(detected_url, 'urlplaceholder')
    # replace everything that's not a space or a a
    text = re.sub(r"[^\w\s]"," ", text)
    
    tokens = nltk.word_tokenize(text)
    tokens = [w.lower().strip() for w in tokens if w not in stopwords_eng]

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [47]:
Y = Y.loc[:,~(Y.sum() == 0).to_numpy()]
X_train, X_test, y_train, y_test = train_test_split(X, Y)

pipeline  = Pipeline(steps=[
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('moc', MultiOutputClassifier(GradientBoostingClassifier()))
    ], verbose=True)
# train classifier
pipeline.fit(X_train, y_train)

# # predict on test data
y_pred = pipeline.predict(X_test)

# # display results



/Users/harryashby/anaconda3/envs/u-ds-2/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[Pipeline] .............. (step 1 of 3) Processing vect, total=  30.8s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing moc, total= 6.3min


In [64]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline


# Define your pipeline
pipeline  = Pipeline(steps=[
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('moc', MultiOutputClassifier(GradientBoostingClassifier()))
])

# Define a parameter grid to search over
param_grid = {
    # Parameters for CountVectorizer
    'vect__max_df': [0.75, 0.9],  # Ignore words that appear in more than 75% or 100% of documents
    'vect__ngram_range': [(1, 1), (1, 2)],  # Use unigrams and bigrams
    
    # Parameters for GradientBoostingClassifier inside MultiOutputClassifier
    'moc__estimator__n_estimators': [50, 100],  # Number of boosting stages
    'moc__estimator__learning_rate': [0.1, 0.01],  # Learning rate for boosting
    'moc__estimator__max_depth': [3, 5],  # Maximum depth of trees
}

# Set up GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=3, verbose=3, n_jobs=-1)

# Fit grid search on training data
grid_search.fit(X_train, y_train)

# Get the best parameters and the best score
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation score: {:.3f}".format(grid_search.best_score_))

# Use the best model to predict on test data
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

Fitting 3 folds for each of 32 candidates, totalling 96 fits


/Users/harryashby/anaconda3/envs/u-ds-2/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/harryashby/anaconda3/envs/u-ds-2/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/harryashby/anaconda3/envs/u-ds-2/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/harryashby/anaconda3/envs/u-ds-2/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/harryashby/anaconda3/envs/u-ds-2/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parame

[CV 1/3] END moc__estimator__learning_rate=0.1, moc__estimator__max_depth=3, moc__estimator__n_estimators=50, vect__max_df=0.9, vect__ngram_range=(1, 1);, score=0.235 total time= 4.0min
[CV 3/3] END moc__estimator__learning_rate=0.1, moc__estimator__max_depth=3, moc__estimator__n_estimators=50, vect__max_df=0.75, vect__ngram_range=(1, 1);, score=0.228 total time= 4.0min
[CV 1/3] END moc__estimator__learning_rate=0.1, moc__estimator__max_depth=3, moc__estimator__n_estimators=50, vect__max_df=0.75, vect__ngram_range=(1, 1);, score=0.236 total time= 4.0min


/Users/harryashby/anaconda3/envs/u-ds-2/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/harryashby/anaconda3/envs/u-ds-2/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/harryashby/anaconda3/envs/u-ds-2/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 2/3] END moc__estimator__learning_rate=0.1, moc__estimator__max_depth=3, moc__estimator__n_estimators=50, vect__max_df=0.75, vect__ngram_range=(1, 1);, score=0.221 total time= 4.1min


...

[CV 1/3] END moc__estimator__learning_rate=0.01, moc__estimator__max_depth=5, moc__estimator__n_estimators=100, vect__max_df=0.9, vect__ngram_range=(1, 1);, score=0.221 total time= 8.6min


/Users/harryashby/anaconda3/envs/u-ds-2/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 2/3] END moc__estimator__learning_rate=0.01, moc__estimator__max_depth=5, moc__estimator__n_estimators=100, vect__max_df=0.9, vect__ngram_range=(1, 1);, score=0.218 total time= 8.7min
[CV 3/3] END moc__estimator__learning_rate=0.01, moc__estimator__max_depth=5, moc__estimator__n_estimators=100, vect__max_df=0.9, vect__ngram_range=(1, 1);, score=0.225 total time= 8.6min
[CV 1/3] END moc__estimator__learning_rate=0.01, moc__estimator__max_depth=5, moc__estimator__n_estimators=100, vect__max_df=0.75, vect__ngram_range=(1, 2);, score=0.222 total time=25.9min
[CV 2/3] END moc__estimator__learning_rate=0.01, moc__estimator__max_depth=5, moc__estimator__n_estimators=100, vect__max_df=0.75, vect__ngram_range=(1, 2);, score=0.218 total time=26.0min
[CV 3/3] END moc__estimator__learning_rate=0.01, moc__estimator__max_depth=5, moc__estimator__n_estimators=100, vect__max_df=0.75, vect__ngram_range=(1, 2);, score=0.222 total time=25.2min
[CV 1/3] END moc__estimator__learning_rate=0.01, moc__est

/Users/harryashby/anaconda3/envs/u-ds-2/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Best parameters found:  {'moc__estimator__learning_rate': 0.1, 'moc__estimator__max_depth': 5, 'moc__estimator__n_estimators': 100, 'vect__max_df': 0.9, 'vect__ngram_range': (1, 1)}
Best cross-validation score: 0.240


In [65]:
import pickle    
pickle.dump(best_model, open('bm.pkl', 'wb'))

In [38]:
y_train.loc[:,(y_train.sum() ==0).to_list()]

,child_alone
18911,0
11376,0
17973,0
25207,0
7485,0
...,...
15193,0
19628,0
12584,0
5796,0


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [66]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

def evaluate_multi_output(y_test, y_pred):
    """
    Evaluates a multi-output classifier using precision, recall, f1, and accuracy metrics.
    
    Parameters:
    y_test (array-like): True labels, shape (n_samples, n_outputs)
    y_pred (array-like): Predicted labels, shape (n_samples, n_outputs)
    
    Returns:
    dict: A dictionary with average precision, recall, f1, and accuracy across all outputs.
    """
    # Initialize lists to store scores for each output
    precision_list = []
    recall_list = []
    f1_list = []
    accuracy_list = []

    # Iterate over each output (column in y_test and y_pred)
    for i in range(y_test.shape[1]):
        # For each output label, compute precision, recall, f1, and accuracy
        precision = precision_score(y_test[:, i], y_pred[:, i], average='weighted', zero_division=0)
        recall = recall_score(y_test[:, i], y_pred[:, i], average='weighted', zero_division=0)
        f1 = f1_score(y_test[:, i], y_pred[:, i], average='weighted', zero_division=0)
        accuracy = accuracy_score(y_test[:, i], y_pred[:, i])
        
        # Append to respective lists
        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)
        accuracy_list.append(accuracy)

    # Calculate average scores across all outputs
    average_precision = sum(precision_list) / len(precision_list)
    average_recall = sum(recall_list) / len(recall_list)
    average_f1 = sum(f1_list) / len(f1_list)
    average_accuracy = sum(accuracy_list) / len(accuracy_list)

    # Return results as a dictionary
    return {
        'average_precision': average_precision,
        'average_recall': average_recall,
        'average_f1': average_f1,
        'average_accuracy': average_accuracy
    }


evaluate_multi_output(y_test.to_numpy(), y_pred)


{'average_precision': 0.9386807629716255,
 'average_recall': 0.9449540084572127,
 'average_f1': 0.9381061439179247,
 'average_accuracy': 0.9449540084572127}

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.